<a href="https://colab.research.google.com/github/RafaelCaballero/Julio25/blob/main/code/proyecto_regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python

## Regresión sobre datos bursátiles

### Rafael Caballero

#### Paso 0: importar librerías y cargar y preprocesar datos

In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer
from math import sqrt
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf
import pandas as pd

# Lista de tickers:
valores = ["inditex","iberdrola", "santander", "BBVA", "naturgy"]
tickers = ['ITX.MC', 'IBE.MC',    'SAN.MC', 'BBVA.MC', 'NTGY.MC']

# Fechas de inicio y fin
start_date = '2022-01-01'
end_date = '2025-07-07'

# Descargar los datos de Yahoo Finance
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# Crear un DataFrame vacío para almacenar los valores OPEN
close_data = pd.DataFrame()

# Extraer los valores OPEN de cada ticker y añadirlos al DataFrame
for i,ticker in enumerate(tickers):
    nombre = valores[i]
    close_data[nombre] = data[ticker]['Close']


# incremento en porcentaje con respecto al día anterior
roi = close_data.pct_change()*100
roi.columns = ["r_"+c for c in close_data.columns]

df = pd.concat([close_data,roi],axis=1)

# añadimos el día de la semana
df['dia_semana'] = roi.index.to_series().dt.day_name()

# mejor en castellano
days_translation = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
df['dia_semana'] = df['dia_semana'].map(days_translation)

# Establecer el índice como una versión formateada de la fecha
df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')

df


/tmp/ipython-input-11-4278836117.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
[*********************100%***********************]  5 of 5 completed


,inditex,iberdrola,santander,BBVA,naturgy,r_inditex,r_iberdrola,r_santander,r_BBVA,r_naturgy,dia_semana
Date,,,,,,,,,,,
2022-01-03,26.062571,8.914805,2.562269,4.118002,23.679724,NaN,NaN,NaN,NaN,NaN,Lunes
2022-01-04,25.926165,8.863596,2.633479,4.222127,23.588680,-0.523379,-0.574431,2.779202,2.528532,-0.384479,Martes
2022-01-05,25.880697,8.803850,2.637821,4.211949,23.621788,-0.175373,-0.674058,0.164880,-0.241065,0.140354,Miércoles
2022-01-06,25.707914,8.748374,2.679505,4.273015,23.737661,-0.667613,-0.630136,1.580240,1.449808,0.490536,Jueves
2022-01-07,25.153198,8.752640,2.702953,4.374008,23.605234,-2.157764,0.048761,0.875060,2.363511,-0.557878,Viernes
...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,44.180000,15.887238,7.027000,13.055000,27.000000,0.249604,-0.214391,-0.142109,0.345888,0.671142,Lunes
2025-07-01,44.369999,15.979888,6.965000,12.850000,27.219999,0.430056,0.583175,-0.882308,-1.570279,0.814812,Martes
2025-07-02,43.990002,15.887238,7.120000,13.035000,27.160000,-0.856428,-0.579794,2.225409,1.439685,-0.220424,Miércoles


In [12]:
df = df.iloc[1:]
df

,inditex,iberdrola,santander,BBVA,naturgy,r_inditex,r_iberdrola,r_santander,r_BBVA,r_naturgy,dia_semana
Date,,,,,,,,,,,
2022-01-04,25.926165,8.863596,2.633479,4.222127,23.588680,-0.523379,-0.574431,2.779202,2.528532,-0.384479,Martes
2022-01-05,25.880697,8.803850,2.637821,4.211949,23.621788,-0.175373,-0.674058,0.164880,-0.241065,0.140354,Miércoles
2022-01-06,25.707914,8.748374,2.679505,4.273015,23.737661,-0.667613,-0.630136,1.580240,1.449808,0.490536,Jueves
2022-01-07,25.153198,8.752640,2.702953,4.374008,23.605234,-2.157764,0.048761,0.875060,2.363511,-0.557878,Viernes
2022-01-10,25.171387,8.696228,2.705558,4.366179,23.770771,0.072311,-0.644511,0.096392,-0.178993,0.701272,Lunes
...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,44.180000,15.887238,7.027000,13.055000,27.000000,0.249604,-0.214391,-0.142109,0.345888,0.671142,Lunes
2025-07-01,44.369999,15.979888,6.965000,12.850000,27.219999,0.430056,0.583175,-0.882308,-1.570279,0.814812,Martes
2025-07-02,43.990002,15.887238,7.120000,13.035000,27.160000,-0.856428,-0.579794,2.225409,1.439685,-0.220424,Miércoles


In [13]:
df.to_excel("datos_preparados.xlsx",index=False)

In [14]:
df["r_BBVA_mañana"] = df["r_BBVA"].shift(-1)
df

/tmp/ipython-input-14-3679338387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["r_BBVA_mañana"] = df["r_BBVA"].shift(-1)


,inditex,iberdrola,santander,BBVA,naturgy,r_inditex,r_iberdrola,r_santander,r_BBVA,r_naturgy,dia_semana,r_BBVA_mañana
Date,,,,,,,,,,,,
2022-01-04,25.926165,8.863596,2.633479,4.222127,23.588680,-0.523379,-0.574431,2.779202,2.528532,-0.384479,Martes,-0.241065
2022-01-05,25.880697,8.803850,2.637821,4.211949,23.621788,-0.175373,-0.674058,0.164880,-0.241065,0.140354,Miércoles,1.449808
2022-01-06,25.707914,8.748374,2.679505,4.273015,23.737661,-0.667613,-0.630136,1.580240,1.449808,0.490536,Jueves,2.363511
2022-01-07,25.153198,8.752640,2.702953,4.374008,23.605234,-2.157764,0.048761,0.875060,2.363511,-0.557878,Viernes,-0.178993
2022-01-10,25.171387,8.696228,2.705558,4.366179,23.770771,0.072311,-0.644511,0.096392,-0.178993,0.701272,Lunes,0.322742
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,44.180000,15.887238,7.027000,13.055000,27.000000,0.249604,-0.214391,-0.142109,0.345888,0.671142,Lunes,-1.570279
2025-07-01,44.369999,15.979888,6.965000,12.850000,27.219999,0.430056,0.583175,-0.882308,-1.570279,0.814812,Martes,1.439685
2025-07-02,43.990002,15.887238,7.120000,13.035000,27.160000,-0.856428,-0.579794,2.225409,1.439685,-0.220424,Miércoles,1.687766
